## FairAC Experiments on the Pokec-N dataset
This notebook can be used to run FairAC experiments on the Pokec-N dataset.

It is currently configured to run a full training run on three different seeds.

In [1]:
import sys
import os
import torch
sys.path.append(os.path.abspath("../"))
from base_experiment import ExperimentRunner

### Set up the experiment runner
First we create an experiment runner, which is used to set the random seeds and provide params/logging directories to the different runs.

In [2]:
# Set up the experiment runner with the all the seeds and params we want
experiment = ExperimentRunner(
    experiment_name = "pokec_n_fair_ac_main",
    seeds = [40, 41, 42],
    data_path = "dataset/pokec",
    log_dir="experiments/fair_ac/logs/pokec_n", 
    device=1,
    params=[{"lambda1": 1.0, "lambda2": 0.5}] # for pokec-n the original work used lambda2 = 0.5
)

# After we set up the experiment, we can import the rest
from dataset import PokecN
from models.gnn import WrappedGNNConfig
from models.fair.ac import FairAC, Trainer

FairAC wraps a GNN with a sensitive classifier for the downstream task, so we configure the GNN + sensitive classifier combo for the downstream task


In [3]:
gnn_config = WrappedGNNConfig(
    hidden_dim=128,
    kind="GCN",
    lr=1e-3,
    weight_decay=1e-5,
    kwargs={"dropout": 0.5},
)

In [4]:
for (seed, log_dir, device, params) in experiment.runs():
    print("===========================")
    print(f"Running {experiment.experiment_name} using seed {seed}")
    print(f"Log directory: {log_dir}")
    print(f"Params: {params}")
    print("===========================")
    
    # Load in the dataset
    dataset = PokecN(
        nodes_path=experiment.data_path / "region_job_2.csv",
        edges_path=experiment.data_path / "region_job_2_relationship.txt",
        embedding_path=experiment.data_path / "pokec_n_embedding10.npy",
        feat_drop_rate=0.3,
        device=device
    )

    print(f"Loaded dataset with {dataset.graph.num_nodes()} nodes and {dataset.graph.num_edges()} edges")
    print(f"Using feat_drop_rate: {dataset.feat_drop_rate}")

    # Create FairAC model
    fair_ac = FairAC(
        feature_dim=dataset.features.shape[1],
        transformed_feature_dim=128,
        emb_dim=dataset.embeddings.shape[1],
        attn_vec_dim=128,
        attn_num_heads=1,
        dropout=0.5,
        num_sensitive_classes=1,
    ).to(device)
    print(f"Created FairAC model with {1} sensitive class")
        
    # Create FairAC trainer
    trainer = Trainer(
        ac_model=fair_ac,
        lambda1=params["lambda1"],
        lambda2=params["lambda2"],
        dataset=dataset,
        device=device,
        gnn_config=gnn_config,
        log_dir=log_dir,
        min_acc=0.65,
        min_roc=0.69,
    )
    print(f"Created trainer with {'GCN'} model, using log_dir: {log_dir}")

    print("Starting pre-training phase")
    # Run pre-training
    trainer.pretrain(epochs=200)
    print("Finished pretraining")
    
    # Main training loop, with GNN validation
    print("Starting main training...")
    trainer.train(val_start_epoch=800, val_epoch_interval=200, epochs=2800)
    # As we allocate the entire dataset on the gpu, we need to de-allocate it, before starting over.
    del dataset
    del trainer
    del fair_ac
    torch.cuda.empty_cache()
    print("Cleared cuda cache")


Running pokec_n_fair_ac_main using seed 40
Log directory: /home/fact21/fact_refactor/experiments/fair_ac/logs/pokec_n/pokec_n_fair_ac_main_40_lambda1_1.0_lambda2_0.5
Params: {'lambda1': 1.0, 'lambda2': 0.5}
Loaded dataset with 66569 nodes and 1100663 edges
Using feat_drop_rate: 0.3
Created FairAC model with 1 sensitive class
Created trainer with GCN model, using log_dir: /home/fact21/fact_refactor/experiments/fair_ac/logs/pokec_n/pokec_n_fair_ac_main_40_lambda1_1.0_lambda2_0.5
Starting pre-training phase


  0%|          | 0/200 [00:00<?, ?it/s]

Finished pretraining
Starting main training...


  0%|          | 0/2800 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Finished training!
Best epoch: 1600

Best fair model:
	acc: 0.6532
	roc: 0.7101
	parity: 0.0007
	equality: 0.0100
	consistency: 0.4597

Best acc model:
	acc: 0.6932
	roc: 0.7155
	parity: 0.0594
	equality: 0.0645

Best auc model:
	acc: 0.6855
	roc: 0.7199
	parity: 0.0450
	equality: 0.0419
Cleared cuda cache
